In [10]:
using LinearAlgebraicRepresentation
Lar = LinearAlgebraicRepresentation
using BenchmarkTools

## Funzione da ottimizzare

In [11]:
function boundingbox(vertices::Lar.Points)
   minimum = mapslices(x->min(x...), vertices, dims=2)
   maximum = mapslices(x->max(x...), vertices, dims=2)
   return minimum, maximum
end

boundingbox (generic function with 1 method)

## Dati in input

In [12]:
b=[[],[]]
EV=[[1,1]]

for i=1:60
           push!(b[1],(1.0 + i*2.0))
           push!(b[2],(1.0 + i*2.0))
           push!(b[1],(4.0 + i*2.0))
           push!(b[2],(1.0 + i*2.0))
           push!(b[1],(1.0 + i*2.0))
           push!(b[2],(4.0 + i*2.0))
           push!(b[1],(4.0 + i*2.0))
           push!(b[2],(4.0 + i*2.0))
           push!(EV,[1+4*(i-1),2+4*(i-1)])
           push!(EV,[1+4*(i-1),3+4*(i-1)])
           push!(EV,[2+4*(i-1),4+4*(i-1)])
           push!(EV,[3+4*(i-1),4+4*(i-1)])
end

V = permutedims(reshape(hcat(b...), (length(b[1]), length(b))))
filter!(e->e!=[1,1],EV)

240-element Array{Array{Int64,1},1}:
 [1, 2]
 [1, 3]
 [2, 4]
 [3, 4]
 [5, 6]
 [5, 7]
 [6, 8]
 [7, 8]
 [9, 10]
 [9, 11]
 [10, 12]
 [11, 12]
 [13, 14]
 ⋮
 [229, 230]
 [229, 231]
 [230, 232]
 [231, 232]
 [233, 234]
 [233, 235]
 [234, 236]
 [235, 236]
 [237, 238]
 [237, 239]
 [238, 240]
 [239, 240]

## 0) Benchmark vecchia funzione

In [13]:
@btime boundingbox(V)

  81.438 μs (2029 allocations: 46.78 KiB)


([3.0; 3.0], [124.0; 124.0])

## 1) Controllo se la funzione è type unstable

In [14]:
@code_warntype boundingbox(V)

Variables
  #self#::Core.Compiler.Const(boundingbox, false)
  vertices::Array{Float64,2}
  #17::var"#17#19"
  #18::var"#18#20"
  minimum::Any
  maximum::Any

Body::Tuple{Any,Any}
1 ─       (#17 = %new(Main.:(var"#17#19")))
│   %2  = #17::Core.Compiler.Const(var"#17#19"(), false)
│   %3  = (:dims,)::Core.Compiler.Const((:dims,), false)
│   %4  = Core.apply_type(Core.NamedTuple, %3)::Core.Compiler.Const(NamedTuple{(:dims,),T} where T<:Tuple, false)
│   %5  = Core.tuple(2)::Core.Compiler.Const((2,), false)
│   %6  = (%4)(%5)::Core.Compiler.Const((dims = 2,), false)
│   %7  = Core.kwfunc(Main.mapslices)::Core.Compiler.Const(Base.var"#mapslices##kw"(), false)
│         (minimum = (%7)(%6, Main.mapslices, %2, vertices))
│         (#18 = %new(Main.:(var"#18#20")))
│   %10 = #18::Core.Compiler.Const(var"#18#20"(), false)
│   %11 = (:dims,)::Core.Compiler.Const((:dims,), false)
│   %12 = Core.apply_type(Core.NamedTuple, %11)::Core.Compiler.Const(NamedTuple{(:dims,),T} where T<:Tuple, false)
│  

La funzione NON è type unstable in quanto ritorna la stringa

    Body::Tuple{Any,Any}

## Parallelizzazione codice e uso dei threads  
  
La funzione originaria utilizzava la funzione 'mapslices', che non è parallelizzabile. Abbiamo quindi riscritto completamente la funzione in modo da renderla accessibile alla macro @threads.

In [19]:
using Base.Threads
function boundingbox2(vertices::Lar.Points)
       d=size(V)[1]
       numPoints=size(V)[2]
       mins::Array{Float64,1}=[]
       maxs::Array{Float64,1}=[]
       @threads for axis=1:d
               row = V[d,:]
               min=V[d]
               max=V[d]
               @threads for points=2:numPoints
                       if(row[points]<min)
                               min = row[points]
                       end
                       if(row[points]>max)
                               max = row[points]
                       end
               end
               push!(mins,min)
               push!(maxs,max)
       end
  return mins, maxs
end
print("Numero di threads allocati :")
println(nthreads())
@btime boundingbox2(V)

Numero di threads allocati :2
  25.978 μs (1115 allocations: 23.22 KiB)


([3.0, 3.0], [124.0, 124.0])